# Scenario 4 : inter_square

## library

In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

required_packages = ['base', 'forecast', 'CondIndTests','RCIT'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

In [2]:
r = robjects.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')

In [3]:
RCIT = importr('RCIT')

In [4]:
pcalg = importr('pcalg')

In [5]:
def IgnorTest(cause=None, effect=None, explorer=None, covariate=None, level=0.05, dtype="continuous"): #inputs are arrays, dtypes are continuous/binary/mixed
    tmp_cause=pd.DataFrame({'A': cause})
    tmp_effect=pd.DataFrame({'Y': effect})
    tmp_explorer=pd.DataFrame({'V': explorer})
    tmp_covariate = pd.DataFrame(covariate)
    dim_covariate = tmp_covariate.shape[1]
    
    if dtype=="continuous":
        if dim_covariate==0:
            tmp_conditioning=tmp_cause
        else:
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)

        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_conditioning = ro.conversion.py2rpy(tmp_conditioning)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_effect = ro.conversion.py2rpy(tmp_effect)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_explorer = ro.conversion.py2rpy(tmp_explorer)
            
        sample_size = r.unlist(r.dim(r_from_pd_conditioning))[0]
        dim = r.unlist(r.dim(r_from_pd_conditioning))[1]
            
        r_from_pd_explorer=r.unlist(r_from_pd_explorer)    
        r_from_pd_effect=r.unlist(r_from_pd_effect)   
        r_from_pd_conditioning=r.unlist(r_from_pd_conditioning)   
        r_from_pd_conditioning=r.matrix(r_from_pd_conditioning,sample_size,dim)
        
        #test=r.CondIndTest(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning, method = "KCI")
        #pvalue=r.unlist(test)[3]
        test=r.RCoT(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning,seed=2)
        pvalue=r.unlist(test)[0]
        print(pvalue)
           
    elif dtype=="binary":
        if covariate==None:
            tmp_conditioning=tmp_cause
        else:
            tmp_covariate=pd.DataFrame(covariate)
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)
            
    elif dtype=="mixed":
        pass
                        
    else:
        raise ValueError("explorertype is not valid.")
        
    if(pvalue<=level):
        print("The Ignorability Assumption is not satisfied.")
    else:
        print("The Ignorability Assumption is satisfied.")

    return pvalue

### parameter setting

In [6]:
n = 4000 #changeable
c1 = 1
c2 = 1 
c3 = 1 #changeable
c4 = 1
c5 = 1
r1 = r2 = r3 = r4 = 1 #changeable
import scipy.stats as stats

### Simulation

In [7]:
#Figure 1
np.random.seed(1)

COUNT1=0 #기각 횟수
for i in range(1000):

    U = np.random.normal(0,1,n) #common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+X1**2+ey
    
    if i==1:
        f1_cor12=stats.pearsonr(X1, X2)[0]
    
    pvalue = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous")
    if pvalue<=0.05:
        COUNT1 = COUNT1+1

0.6537275006858353
The Ignorability Assumption is satisfied.
0.0990690988670031
The Ignorability Assumption is satisfied.
0.521282385438044
The Ignorability Assumption is satisfied.
0.32595070756014277
The Ignorability Assumption is satisfied.
0.7196315186788679
The Ignorability Assumption is satisfied.
0.9780281289920206
The Ignorability Assumption is satisfied.
0.4432840412943655
The Ignorability Assumption is satisfied.
0.961944279096876
The Ignorability Assumption is satisfied.
0.785741656028426
The Ignorability Assumption is satisfied.
0.5057684038983759
The Ignorability Assumption is satisfied.
0.909580457440635
The Ignorability Assumption is satisfied.
0.2204669938481083
The Ignorability Assumption is satisfied.
0.5845945525904936
The Ignorability Assumption is satisfied.
0.877649045456171
The Ignorability Assumption is satisfied.
0.32921120779749213
The Ignorability Assumption is satisfied.
0.3887221289581584
The Ignorability Assumption is satisfied.
0.13698967340954116
The Ign

In [8]:
#Figure 2
np.random.seed(1)

COUNT21=0 #기각 횟수
COUNT22=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = c1*X1+c3*C+X1*C+X1**2+C**2+ey
    
    if i==1:
        f2_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT21 = COUNT21+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT22 = COUNT22+1 #2종 오류

0.3700516531214104
The Ignorability Assumption is satisfied.
0.0008137191007195899
The Ignorability Assumption is not satisfied.
0.7524980901552658
The Ignorability Assumption is satisfied.
2.1444624831445935e-07
The Ignorability Assumption is not satisfied.
0.06725047201176515
The Ignorability Assumption is satisfied.
1.4666833771936894e-05
The Ignorability Assumption is not satisfied.
0.46268652369422003
The Ignorability Assumption is satisfied.
1.5051590517067837e-05
The Ignorability Assumption is not satisfied.
0.32882461865578216
The Ignorability Assumption is satisfied.
1.9819719524472035e-06
The Ignorability Assumption is not satisfied.
0.7413692023404294
The Ignorability Assumption is satisfied.
7.42783580198747e-05
The Ignorability Assumption is not satisfied.
0.248575145449765
The Ignorability Assumption is satisfied.
1.8173090909900935e-07
The Ignorability Assumption is not satisfied.
0.1588234552729535
The Ignorability Assumption is satisfied.
0.011776582218294473
The Ignor

In [9]:
#Figure 3
np.random.seed(1)

COUNT31=0 #기각 횟수
COUNT32=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = c3*C+C**2+ey
    
    if i==1:
        f3_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT31 = COUNT31+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT32 = COUNT32+1 #2종 오류

0.7197610643915426
The Ignorability Assumption is satisfied.
1.0769163338864018e-14
The Ignorability Assumption is not satisfied.
0.5288112062284513
The Ignorability Assumption is satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.888555193803343
The Ignorability Assumption is satisfied.
9.992007221626409e-16
The Ignorability Assumption is not satisfied.
0.2984504177163839
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.7896290061307532
The Ignorability Assumption is satisfied.
1.5543122344752192e-15
The Ignorability Assumption is not satisfied.
0.4726102106421046
The Ignorability Assumption is satisfied.
1.966913298900863e-10
The Ignorability Assumption is not satisfied.
0.9408565180377845
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8510968201373774
The Ignorability Assumption is satisfied.
2.733924198139448e-12
The Ignorability Assumption is

In [10]:
#Figure 4 
np.random.seed(1)

COUNT41=0 #기각 횟수
COUNT42=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = c1*X1+c4*X2+c3*C+X1*X2+X1*C+X2*C+X1**2+X2**2+C**2+ey
    
    if i==1:
        f4_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT41 = COUNT41+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT42 = COUNT42+1 #2종 오류

0.6253426523241552
The Ignorability Assumption is satisfied.
0.01641431076500499
The Ignorability Assumption is not satisfied.
0.314414165445698
The Ignorability Assumption is satisfied.
0.008525646095167483
The Ignorability Assumption is not satisfied.
0.48925410988021334
The Ignorability Assumption is satisfied.
0.03202805273917664
The Ignorability Assumption is not satisfied.
0.35271123327335463
The Ignorability Assumption is satisfied.
0.003157986104848076
The Ignorability Assumption is not satisfied.
0.6138177932781282
The Ignorability Assumption is satisfied.
0.03500764350301455
The Ignorability Assumption is not satisfied.
0.2767367165150263
The Ignorability Assumption is satisfied.
0.005960742220505799
The Ignorability Assumption is not satisfied.
0.4763709944400073
The Ignorability Assumption is satisfied.
0.4136058551485601
The Ignorability Assumption is satisfied.
0.6646732589013445
The Ignorability Assumption is satisfied.
0.0002722322143202671
The Ignorability Assumption i

In [11]:
#Figure 5
np.random.seed(1)

COUNT51=0 #기각 횟수
COUNT52=0
COUNT53=0
COUNT54=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = c1*X1+c3*C+X1*C+X1**2+C**2+ey
    
    if i==1:
        f5_cor12=r1*r2/np.sqrt(r1**2+c2**2+1)/np.sqrt(r2**2+1)
        f5_cor13=r1*r3/np.sqrt(r1**2+c2**2+1)/np.sqrt(r3**2+1)
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT51 = COUNT51+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT52 = COUNT52+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT53 = COUNT53+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT54 = COUNT54+1 #2종 오류

0.05566673057946214
The Ignorability Assumption is satisfied.
0.0023067726017222956
The Ignorability Assumption is not satisfied.
0.019114248856783478
The Ignorability Assumption is not satisfied.
0.00033140367394768333
The Ignorability Assumption is not satisfied.
0.15675495716772136
The Ignorability Assumption is satisfied.
0.00019556076917970344
The Ignorability Assumption is not satisfied.
0.005570495374722539
The Ignorability Assumption is not satisfied.
1.9594437797865183e-06
The Ignorability Assumption is not satisfied.
0.5934728483838253
The Ignorability Assumption is satisfied.
3.5841870922315167e-06
The Ignorability Assumption is not satisfied.
0.9825999788673392
The Ignorability Assumption is satisfied.
1.014579020686135e-06
The Ignorability Assumption is not satisfied.
0.6545691341047548
The Ignorability Assumption is satisfied.
0.018224173347545114
The Ignorability Assumption is not satisfied.
0.5399925439527158
The Ignorability Assumption is satisfied.
3.732144507162527e-

In [12]:
print(f5_cor12)
print(f5_cor13)

0.408248290463863
0.408248290463863


In [13]:
COUNT51

48

In [14]:
COUNT52

999

In [15]:
COUNT53

69

In [16]:
COUNT54

994

In [17]:
#Figure 6
np.random.seed(1)

COUNT61=0 #기각 횟수
COUNT62=0
COUNT63=0
COUNT64=0
for i in range(1000):
    U1 = np.random.normal(0,1,n) #common cause of X1 and X2
    U2 = np.random.normal(0,1,n) #common cause of X1 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U1+c2*C+r3*U2+e1
    X2 = r2*U1+e2
    X3 = r4*U2+e3
    Y = c1*X1+c3*C+X1*C+X1**2+C**2+ey
    
    if i==1:
        f6_cor12=r1*r2/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r2**2+1)
        f6_cor13=r3*r4/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r4**2+1)

    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT61 = COUNT61+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT62 = COUNT62+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT63 = COUNT63+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT64 = COUNT64+1 #2종 오류

0.30100116545010014
The Ignorability Assumption is satisfied.
2.0567271688864963e-05
The Ignorability Assumption is not satisfied.
0.48581680587983467
The Ignorability Assumption is satisfied.
0.12593525978360198
The Ignorability Assumption is satisfied.
0.9039133102478649
The Ignorability Assumption is satisfied.
0.3050933625306913
The Ignorability Assumption is satisfied.
0.6026724970787507
The Ignorability Assumption is satisfied.
0.00913081405938232
The Ignorability Assumption is not satisfied.
0.7335440226589867
The Ignorability Assumption is satisfied.
0.00045769155824326724
The Ignorability Assumption is not satisfied.
0.30909657455744577
The Ignorability Assumption is satisfied.
0.009650884574657015
The Ignorability Assumption is not satisfied.
0.9020973159865974
The Ignorability Assumption is satisfied.
0.01346562936866813
The Ignorability Assumption is not satisfied.
0.7525858220961386
The Ignorability Assumption is satisfied.
0.010934164435535854
The Ignorability Assumption 

In [18]:
print(f6_cor12)
print(f6_cor13)

0.35355339059327373
0.35355339059327373


In [19]:
COUNT61

49

In [20]:
COUNT62

764

In [21]:
COUNT63

61

In [22]:
COUNT64

773

In [23]:
#Figure 7
np.random.seed(1)

COUNT71=0 #기각 횟수
COUNT72=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder
    eu = np.random.normal(0,1,n)
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    U = c2*C+eu #common cause of X1 and X2
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+c3*C+X1*C+X1**2+C**2+ey
    
    if i==1:
        f7_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT71 = COUNT71+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT72 = COUNT72+1 #2종 오류

0.6139864266897326
The Ignorability Assumption is satisfied.
1.454392162258955e-13
The Ignorability Assumption is not satisfied.
0.9160561374628778
The Ignorability Assumption is satisfied.
1.163513729807164e-13
The Ignorability Assumption is not satisfied.
0.11433189338547967
The Ignorability Assumption is satisfied.
8.322672422345079e-06
The Ignorability Assumption is not satisfied.
0.8369013430265589
The Ignorability Assumption is satisfied.
5.937660363386499e-10
The Ignorability Assumption is not satisfied.
0.1201661025868328
The Ignorability Assumption is satisfied.
3.0781873753271682e-06
The Ignorability Assumption is not satisfied.
0.11147249524697633
The Ignorability Assumption is satisfied.
4.955761356839616e-10
The Ignorability Assumption is not satisfied.
0.2823399425115185
The Ignorability Assumption is satisfied.
6.972200594645983e-14
The Ignorability Assumption is not satisfied.
0.5470514647750068
The Ignorability Assumption is satisfied.
1.5440149159218208e-10
The Ignora

In [24]:
#Figure 8
np.random.seed(1)

COUNT81=0 #기각 횟수
COUNT82=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder, common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*C+e1
    X2 = r2*C+e2
    Y = c1*X1+c3*C+X1*C+X1**2+C**2+ey
    
    if i==1:
        f8_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT81 = COUNT81+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT82 = COUNT82+1 #2종 오류

0.6571380888440884
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.4438526483727051
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.046945819255752363
The Ignorability Assumption is not satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.7817403633127433
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5440048046647303
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.20139066687549345
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.4775674085416266
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8197658128223827
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7702522673797487
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assu

In [25]:
#Figure 9
np.random.seed(1)

COUNT91=0 #기각 횟수
COUNT92=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    Y = c1*X1+c3*C+X1*C+X1**2+C**2+ey
    
    if i==1:
        f9_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT91 = COUNT91+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT92 = COUNT92+1 #2종 오류

0.2782703984218229
The Ignorability Assumption is satisfied.
4.1492365099315975e-12
The Ignorability Assumption is not satisfied.
0.7881538626689495
The Ignorability Assumption is satisfied.
7.187389716722947e-08
The Ignorability Assumption is not satisfied.
0.23138433329355568
The Ignorability Assumption is satisfied.
4.254720020746561e-09
The Ignorability Assumption is not satisfied.
0.919531537457649
The Ignorability Assumption is satisfied.
2.0823928270630176e-07
The Ignorability Assumption is not satisfied.
0.27888641377352086
The Ignorability Assumption is satisfied.
2.4910623452889524e-08
The Ignorability Assumption is not satisfied.
0.8841950003435263
The Ignorability Assumption is satisfied.
4.349427373817605e-09
The Ignorability Assumption is not satisfied.
0.1361601685824838
The Ignorability Assumption is satisfied.
1.0158967445050848e-08
The Ignorability Assumption is not satisfied.
0.18205071789058225
The Ignorability Assumption is satisfied.
2.1094237467877974e-15
The Ign

In [26]:
#Figure 10
np.random.seed(1)

COUNT101=0 #기각 횟수
COUNT102=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    X3 = r3*U+e3
    Y = c1*X1+c3*C+c5*X2+X1*X2+X1*C+X2*C+X1**2+C**2+X2**2+ey
    
    if i==1:
        f10_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT101 = COUNT101+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT102 = COUNT102+1 #2종 오류

0.6071241743380865
The Ignorability Assumption is satisfied.
0.002464459960092591
The Ignorability Assumption is not satisfied.
0.20856544881048322
The Ignorability Assumption is satisfied.
0.0004844437287602199
The Ignorability Assumption is not satisfied.
0.63583223124483
The Ignorability Assumption is satisfied.
2.823092381976089e-05
The Ignorability Assumption is not satisfied.
0.86955638372864
The Ignorability Assumption is satisfied.
0.000660472239281451
The Ignorability Assumption is not satisfied.
0.5947891221466509
The Ignorability Assumption is satisfied.
0.001124849066495237
The Ignorability Assumption is not satisfied.
0.2684799581986904
The Ignorability Assumption is satisfied.
0.0018051909304612757
The Ignorability Assumption is not satisfied.
0.7868311203037142
The Ignorability Assumption is satisfied.
0.010649416891066243
The Ignorability Assumption is not satisfied.
0.7000787169126559
The Ignorability Assumption is satisfied.
6.601850675025389e-07
The Ignorability Assu

In [27]:
table1=pd.DataFrame(np.repeat(-1,70).reshape(10,7))
table1.columns = ["Scenario1","1종오류(X2)","2종오류(X2)","1종오류(X3)","2종오류(X3)","r12","r13"]
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1


In [28]:
table1["Scenario1"]=["Figure"+str(i+1) for i in range(10)]

In [29]:
table1["1종오류(X2)"]=[COUNT1,COUNT21,COUNT31,COUNT41,COUNT51,COUNT61,COUNT71,COUNT81,COUNT91,COUNT101]
table1["2종오류(X2)"]=[-1,COUNT22,COUNT32,COUNT42,COUNT52,COUNT62,COUNT72,COUNT82,COUNT92,COUNT102]
table1["1종오류(X3)"]=[-1,-1,-1,-1,COUNT53,COUNT63,-1,-1,-1,-1]
table1["2종오류(X3)"]=[-1,-1,-1,-1,COUNT54,COUNT64,-1,-1,-1,-1]
table1["r12"]=[f1_cor12,f2_cor12,f3_cor12,-1,f5_cor12,f6_cor12,f7_cor12,f8_cor12,f9_cor12,-1]
table1["r13"]=[-1,-1,-1,f4_cor13,f5_cor13,f6_cor13,-1,-1,-1,f10_cor13]

In [30]:
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,Figure1,65,-1,-1,-1,0.499098,-1.000000
1,Figure2,61,996,-1,-1,0.413865,-1.000000
2,Figure3,50,1000,-1,-1,0.413865,-1.000000
3,Figure4,52,699,-1,-1,-1.000000,0.396688
4,Figure5,48,999,69,994,0.408248,0.408248
5,Figure6,49,764,61,773,0.353553,0.353553
6,Figure7,50,1000,-1,-1,0.664412,-1.000000
7,Figure8,52,1000,-1,-1,0.499098,-1.000000
8,Figure9,72,1000,-1,-1,0.664412,-1.000000
9,Figure10,66,996,-1,-1,-1.000000,0.396688


In [31]:
filename="INTER_SQUARE_c"+str(c3)+"_r"+str(r1)+"_s"+str(n/1000)+".csv"
table1.to_csv("C:\\Users\\minsoo\\Desktop\\실험결과(P)\\"+filename,encoding="euc-kr",index=False)